## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


## Read cleaned data files

### Read cleaned jobs listings file and save in dataframe

In [2]:
cleaned_jobs_listings_df = pd.read_csv("cleaned_jobs_listings.csv")

### Check columns info in jobs listings dataframe

In [3]:
cleaned_jobs_listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_title                3909 non-null   object 
 1   salary_estimate          3909 non-null   object 
 2   job_description          3909 non-null   object 
 3   rating                   3909 non-null   float64
 4   company_name             3909 non-null   object 
 5   location                 3909 non-null   object 
 6   headquarters             3909 non-null   object 
 7   size                     3909 non-null   object 
 8   founded                  3909 non-null   int64  
 9   type_of_ownership        3909 non-null   object 
 10  industry                 3909 non-null   object 
 11  sector                   3909 non-null   object 
 12  revenue                  3909 non-null   object 
 13  competitors              3909 non-null   object 
 14  easy_apply              

## Data Preprocessing

### Define stopwords to append to default stopwords

In [4]:
# append special stopwords in articles to default and common stopwords
new_stopwords = STOPWORDS.union(set(['job', 'skill', 'experience', 'team', 'data', 'use', 'like', 'business', 'work', 'ability', 'let', 'example'
                'need', 'new', 'user', 'opportunity', 'candidate', 'provide', 'company', 'one', 'used', 'need', 
                'see', 'make', 'follow', 'going', 'will', 'want', 'well', 'find', 'give', 'change', 'look', 'first', 'using',
                'know', 'model', 'science', 'think', 'year', 'years', 'looking']))

### Method to preprocess data for articles

In [5]:
# lemmatize = words change to noun
# stemmed = words reduced to root form
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='n')

# Convert a document into a list of tokens.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in new_stopwords and len(token) > 3:
            result.append(lemmatize(token))
    return result

In [6]:
preprocessed_jobs_listings = cleaned_jobs_listings_df['cleaned_job_description'].fillna('').astype(str).map(preprocess)

In [7]:
# implements the concept of a Dictionary – a mapping between words and their integer ids.
jobs_listings_dictionary = corpora.Dictionary(preprocessed_jobs_listings)

In [8]:
# Convert into the bag-of-words (BoW) format
jobs_listings_bow_corpus = [jobs_listings_dictionary.doc2bow(text) for text in preprocessed_jobs_listings]

In [9]:
# Convert into TF-IDF format
jobs_listings_tfidf = models.TfidfModel(jobs_listings_bow_corpus)
jobs_listings_tfidf_corpus = jobs_listings_tfidf[jobs_listings_bow_corpus]

## LSA Model

### Result using LSA model + Bag of words

In [10]:
jobs_listings_bow_lsamodel = LsiModel(jobs_listings_bow_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [11]:
jobs_listings_bow_lsamodel.print_topics(num_words=7)

[(0,
  '0.175*"development" + 0.160*"analysis" + 0.147*"skill" + 0.145*"technology" + 0.138*"information" + 0.137*"project" + 0.137*"solution"'),
 (1,
  '-0.248*"learning" + -0.221*"analytics" + -0.200*"machine" + 0.196*"information" + -0.162*"product" + 0.161*"laboratory" + 0.149*"research"'),
 (2,
  '-0.286*"quantum" + -0.202*"world" + -0.163*"employee" + 0.156*"analysis" + 0.152*"analytics" + 0.144*"project" + 0.139*"equivalent"'),
 (3,
  '-0.349*"equivalent" + -0.231*"education" + -0.221*"learning" + -0.190*"machine" + -0.189*"band" + -0.166*"level" + -0.148*"completed"'),
 (4,
  '0.400*"research" + 0.313*"learning" + 0.247*"machine" + 0.208*"scientist" + -0.166*"information" + 0.158*"laboratory" + -0.152*"management"'),
 (5,
  '-0.262*"development" + 0.218*"information" + 0.178*"learning" + 0.176*"employee" + -0.152*"quantum" + -0.148*"project" + 0.147*"university"'),
 (6,
  '-0.353*"research" + 0.268*"laboratory" + -0.220*"development" + 0.198*"medical" + -0.187*"project" + 0.187

### Result using LSA model + TF-IDF

In [12]:
jobs_listings_tfidf_lsamodel = LsiModel(jobs_listings_tfidf_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [13]:
jobs_listings_tfidf_lsamodel.print_topics(num_words=7)

[(0,
  '0.095*"learning" + 0.094*"analytics" + 0.092*"machine" + 0.083*"product" + 0.081*"client" + 0.080*"customer" + 0.079*"model"'),
 (1,
  '-0.614*"quantum" + -0.191*"qiskit" + -0.178*"progressive" + -0.111*"world" + -0.101*"numerics" + -0.101*"quantumenabled" + -0.098*"reinventing"'),
 (2,
  '0.284*"laboratory" + 0.187*"clinical" + 0.151*"cell" + -0.132*"learning" + -0.131*"machine" + 0.112*"assay" + 0.112*"scientific"'),
 (3,
  '-0.180*"locate" + -0.180*"analyze" + -0.176*"printout" + -0.171*"databasesdata" + -0.158*"qualityacquire" + -0.158*"setsfilter" + -0.157*"systemsidentify"'),
 (4,
  '0.158*"cell" + -0.148*"university" + -0.133*"security" + 0.132*"clinical" + -0.119*"clery" + 0.112*"learning" + 0.112*"machine"'),
 (5,
  '-0.727*"tutor" + -0.488*"tutoring" + -0.176*"student" + -0.131*"choose" + -0.123*"session" + -0.085*"schedule" + -0.081*"tutoringonline"'),
 (6,
  '0.143*"pipeline" + -0.132*"university" + 0.127*"cloud" + 0.127*"tutor" + 0.126*"laboratory" + 0.125*"cell" +

## LDA Model

### Result using LDA model + Bag of words

In [14]:
jobs_listings_bow_ldamodel = gensim.models.LdaMulticore(jobs_listings_bow_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [15]:
jobs_listings_bow_ldamodel.print_topics(num_words=7)

[(0,
  '0.008*"product" + 0.007*"analysis" + 0.006*"including" + 0.006*"project" + 0.005*"development" + 0.005*"service" + 0.005*"customer"'),
 (1,
  '0.006*"product" + 0.006*"development" + 0.006*"technology" + 0.005*"solution" + 0.005*"analytics" + 0.005*"support" + 0.005*"learning"'),
 (2,
  '0.006*"analysis" + 0.006*"learning" + 0.006*"analytics" + 0.005*"development" + 0.005*"skill" + 0.005*"technology" + 0.005*"design"'),
 (3,
  '0.007*"process" + 0.005*"analysis" + 0.005*"product" + 0.005*"service" + 0.005*"development" + 0.004*"skill" + 0.004*"research"'),
 (4,
  '0.005*"management" + 0.005*"technology" + 0.005*"knowledge" + 0.005*"skill" + 0.004*"development" + 0.004*"information" + 0.004*"service"'),
 (5,
  '0.007*"development" + 0.007*"analytics" + 0.006*"skill" + 0.006*"analysis" + 0.005*"product" + 0.005*"management" + 0.005*"scientist"'),
 (6,
  '0.006*"skill" + 0.005*"development" + 0.005*"analysis" + 0.005*"working" + 0.004*"project" + 0.004*"information" + 0.004*"resea

In [16]:
pyLDAvis.gensim_models.prepare(jobs_listings_bow_ldamodel, jobs_listings_bow_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.002002  0.005600       1        1  18.422126
0      0.012193 -0.008164       2        1  13.851977
1      0.001358  0.005916       3        1  12.799754
4     -0.006307 -0.007178       4        1  12.623608
9     -0.005898 -0.003255       5        1  10.311345
5      0.006160  0.004892       6        1   9.447387
7     -0.004808 -0.005894       7        1   7.746692
8     -0.007767  0.006150       8        1   6.083573
3      0.006587  0.004360       9        1   5.162143
6      0.000484 -0.002427      10        1   3.551395, topic_info=          Term         Freq        Total Category  logprob  loglift
413    process  3344.000000  3344.000000  Default  30.0000  30.0000
291      skill  4747.000000  4747.000000  Default  29.0000  29.0000
431   solution  4039.000000  4039.000000  Default  28.0000  28.0000
7     analysis  5083.000000  5083.000000  Default  27.0000  27.0000
129    product  4498.000000  4498.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
197  analytics   106.134624  4192.704153  Topic10  -5.7839  -0.3386
356     design    97.171788  3354.031420  Topic10  -5.8721  -0.2036
253  knowledge    98.395345  3638.743068  Topic10  -5.8596  -0.2726
166       team    88.160298  2407.833413  Topic10  -5.9694   0.0305
98     machine    85.475018  3175.375031  Topic10  -6.0003  -0.2771

[854 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
5992       1  0.055286      accenture
5992       2  0.013822      accenture
5992       3  0.248788      accenture
5992       4  0.304074      accenture
5992       5  0.110572      accenture
...      ...       ...            ...
25493      3  0.709046      yearstech
15593      5  0.364707  zeroemissions
38020      3  0.512684       âbigdata
7370       1  0.679283    âexperience
7373       1  0.673891        âshould

[2295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 5, 10, 6, 8, 9, 4, 7])

### Result using LDA model + TF-IDF

In [17]:
jobs_listings_tfidf_ldamodel = gensim.models.LdaMulticore(jobs_listings_tfidf_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [18]:
jobs_listings_tfidf_ldamodel.print_topics(num_words=7)

[(0,
  '0.001*"product" + 0.001*"learning" + 0.001*"analytics" + 0.001*"client" + 0.001*"machine" + 0.001*"solution" + 0.001*"clinical"'),
 (1,
  '0.001*"product" + 0.001*"learning" + 0.001*"machine" + 0.001*"analytics" + 0.001*"research" + 0.001*"clinical" + 0.001*"engineer"'),
 (2,
  '0.001*"learning" + 0.001*"machine" + 0.001*"statistical" + 0.001*"analytics" + 0.001*"product" + 0.001*"clinical" + 0.001*"database"'),
 (3,
  '0.001*"client" + 0.001*"learning" + 0.001*"analytics" + 0.001*"machine" + 0.001*"research" + 0.001*"model" + 0.001*"management"'),
 (4,
  '0.001*"analytics" + 0.001*"analyst" + 0.001*"product" + 0.001*"bull" + 0.001*"statistical" + 0.001*"client" + 0.001*"database"'),
 (5,
  '0.001*"learning" + 0.001*"machine" + 0.001*"analytics" + 0.001*"model" + 0.001*"engineer" + 0.001*"product" + 0.001*"process"'),
 (6,
  '0.001*"learning" + 0.001*"machine" + 0.001*"product" + 0.001*"customer" + 0.001*"analytics" + 0.001*"client" + 0.001*"research"'),
 (7,
  '0.001*"machine"

In [19]:
dominant_topics_df = pd.DataFrame()
for i, row in enumerate(jobs_listings_tfidf_ldamodel[jobs_listings_tfidf_corpus]):
   row = sorted(row, key=lambda x: (x[1]), reverse=True)
   for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0: # => dominant topic
            wp = jobs_listings_tfidf_ldamodel.show_topic(topic_num)
            topic_keywords = ", ".join([word for word, prop in wp])
            dominant_topics_df = dominant_topics_df.append(
                pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True
            )
        else:
          break
dominant_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

In [20]:
dominant_topics_df["Text"] = cleaned_jobs_listings_df['cleaned_job_description']
dominant_topics_df

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
0,6.0,0.9251,"learning, machine, product, customer, analytic...",about at were on a mission to make booking tr...
1,6.0,0.9192,"learning, machine, product, customer, analytic...",at noom we use scientifically proven methods t...
2,6.0,0.9219,"learning, machine, product, customer, analytic...",decodemhttpswwwdecodemcomdata science manager ...
3,8.0,0.9115,"trading, research, model, statistical, analyti...",seeks a dynamic and driven midlevel data anal...
4,6.0,0.9185,"learning, machine, product, customer, analytic...",director data science 200537descriptionedelma...
...,...,...,...,...
3904,0.0,0.9097,"product, learning, analytics, client, machine,...",about us is a digital transformation consultin...
3905,2.0,0.8969,"learning, machine, statistical, analytics, pro...",job descriptioninterpret data analyze results ...
3906,7.0,0.9344,"machine, learning, analytics, product, researc...",job descriptionthe security analytics data eng...
3907,7.0,0.6572,"machine, learning, analytics, product, researc...",the security analytics data engineer will inte...


In [25]:
dominant_topics_df = dominant_topics_df.sort_values(by=['Perc_Contribution'], ascending=False)
dominant_topics_df.head(50)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
3569,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...
2154,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...
3233,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...
113,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...
2525,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...
1970,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...
361,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...
746,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...
1079,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...
1628,4.0,0.9564,"analytics, analyst, product, bull, statistical...",the us department of the treasury has a distin...


In [21]:
pyLDAvis.gensim_models.prepare(jobs_listings_tfidf_ldamodel, jobs_listings_bow_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.010150  0.004845       1        1  20.899184
9     -0.010696 -0.003176       2        1  20.123168
7     -0.003151 -0.001952       3        1  12.356177
8     -0.003433 -0.005916       4        1  11.945088
2     -0.002266 -0.004686       5        1  10.666314
6     -0.005585  0.008713       6        1  10.565271
5      0.004135  0.006613       7        1   4.803142
4      0.007739 -0.015649       8        1   4.485981
1      0.010040  0.008799       9        1   2.286131
3      0.013366  0.002410      10        1   1.869545, topic_info=            Term        Freq       Total Category  logprob  loglift
93      learning  785.000000  785.000000  Default  30.0000  30.0000
197    analytics  745.000000  745.000000  Default  29.0000  29.0000
338       client  683.000000  683.000000  Default  28.0000  28.0000
98       machine  742.000000  742.000000  Default  27.0000  27.0000
139     research  646.000000  646.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
411     position    7.991830  461.289306  Topic10  -7.7285  -0.0761
499  information    8.265715  538.196402  Topic10  -7.6948  -0.1966
35      customer    8.410650  643.708106  Topic10  -7.6774  -0.3583
37      database    8.165700  538.168105  Topic10  -7.7070  -0.2088
540       system    8.139239  524.249301  Topic10  -7.7102  -0.1858

[805 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
45491      1  0.665003  abercrombie
45491      2  0.073889  abercrombie
45491      3  0.036945  abercrombie
45491      4  0.036945  abercrombie
45491      5  0.036945  abercrombie
...      ...       ...          ...
7373       5  0.083705      âshould
7373       6  0.083705      âshould
7373       7  0.083705      âshould
7373       8  0.083705      âshould
7373      10  0.083705      âshould

[4603 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 10, 8, 9, 3, 7, 6, 5, 2, 4])